In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.express as px

%load_ext autoreload
%autoreload 2

In [2]:
import __init__

from src.actors.actor import Actor

actor = Actor({})


TypeError: ChatMessageHistory.add_message() takes 2 positional arguments but 3 were given

In [8]:
from langchain.memory import ChatMessageHistory

config = {}
config.setdefault("purpose", "You are a helpful assistant")

history = ChatMessageHistory()
history.add_message("system")

history.add_message("Hello", )

TypeError: ChatMessageHistory.add_message() takes 2 positional arguments but 3 were given

In [5]:
config["purpose"]

'You are a helpful assistant'

In [7]:
history

ChatMessageHistory(messages=['system'])